# ECI por alcaldía

Notebook para extrar el índice de complejidad económica por cada alcaldía de la Ciudad de México. El ECI por alcaldía se obtuvo de https://datamexico.org

In [1]:
import pandas as pd
from urllib.request import urlopen
import json

## Lista de alcaldías de la CDMX

In [2]:
alc_df = pd.read_csv('../alcaldias_cdmx.csv')
alc_df.loc[4] = 'Cuajimalpa de Morelos'
alc_df.loc[9] = 'La Magdalena Contreras'
alcaldias = list(alc_df.alcaldia)
print(alcaldias)

['Álvaro Obregón', 'Azcapotzalco', 'Benito Juárez', 'Coyoacán', 'Cuajimalpa de Morelos', 'Cuauhtémoc', 'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras', 'Miguel Hidalgo', 'Milpa Alta', 'Tláhuac', 'Tlalpan', 'Venustiano Carranza', 'Xochimilco']


## Obteniendo los ECI por municipio
El dataframe ```municipios_eci_df``` contiene los índices de complejidad económica con un nivel geográfico de municipio y un nivel industrial basado en el SCIAN para el año 2020 y bajo un indicador de empleados.

In [3]:
eci_url = "https://datamexico.org/api/stats/eci?cube=inegi_denue&Month=20200417,20191114,20190410&rca=Municipality,NAICS%20Industry,Number%20of%20Employees%20Midpoint&threshold=NAICS%20Industry:900,Municipality:900&debug=true&locale=es"
municipios_eci_df = pd.read_json(eci_url)
municipios_eci_df = pd.DataFrame(municipios_eci_df.data.values.tolist())

In [4]:
municipios_eci_df.rename(columns={'Municipality ID': 'municipio_id', 'Number of Employees Midpoint ECI':'ECI', 'Municipality':'municipio'}, inplace=True)
del municipios_eci_df['municipio_id']
municipios_eci_df.head()

,ECI,municipio
0,3.049991,Aguascalientes
1,-0.361775,Asientos
2,0.342401,Calvillo
3,-0.513439,Cosío
4,3.094544,Jesús María


## Filtrando por alcaldías de la CDMX 

In [5]:
alcaldias_eci_df = municipios_eci_df[municipios_eci_df.municipio.isin(alcaldias)]

In [6]:
alcaldias_dict = dict()
for _, row in alcaldias_eci_df.iterrows():
    municipio = row.municipio
    eci = row.ECI
    if municipio not in alcaldias_dict:
        alcaldias_dict[municipio] = [-10e2]
    alcaldias_dict[municipio] = [max(eci, alcaldias_dict[municipio][0])]

alcaldias_names = alcaldias_dict.keys()
eci_values = []
for alcaldia in alcaldias_names:
    eci_values.append(alcaldias_dict[alcaldia][0])

In [7]:
alcaldias_eci_clean_df = pd.DataFrame(data={'alcaldia': alcaldias_names, 'eci': eci_values})
alcaldias_eci_clean_df.loc[0, 'alcaldia'] = 'Cuauhtemoc'
alcaldias_eci_clean_df.loc[3, 'alcaldia'] = 'Coyoacan'
alcaldias_eci_clean_df.loc[10, 'alcaldia'] = 'Alvaro Obregon'
alcaldias_eci_clean_df.loc[11, 'alcaldia'] = 'Tlahuac'
alcaldias_eci_clean_df.loc[14, 'alcaldia'] = 'Benito Juarez'
alcaldias_eci_clean_df

,alcaldia,eci
0,Cuauhtemoc,3.471986
1,Venustiano Carranza,2.501312
2,Azcapotzalco,3.865509
3,Coyoacan,2.800526
4,Cuajimalpa de Morelos,3.854617
5,Gustavo A. Madero,2.680508
6,Iztacalco,2.944861
7,Iztapalapa,2.629184
8,La Magdalena Contreras,1.623270
9,Milpa Alta,0.343307


In [8]:
alcaldias_eci_clean_df.alcaldia = alcaldias_eci_clean_df.alcaldia.str.lower()
alcaldias_eci_clean_df

,alcaldia,eci
0,cuauhtemoc,3.471986
1,venustiano carranza,2.501312
2,azcapotzalco,3.865509
3,coyoacan,2.800526
4,cuajimalpa de morelos,3.854617
5,gustavo a. madero,2.680508
6,iztacalco,2.944861
7,iztapalapa,2.629184
8,la magdalena contreras,1.623270
9,milpa alta,0.343307


In [9]:
alcaldias_eci_clean_df.to_csv('../alcaldias_eci.csv', index=False)